In [1]:
# импорт библиотек
import numpy as np # для матричных вычислений
import pandas as pd # для анализа и предобработки данных

from sklearn import linear_model # линейные моделиё
from sklearn import ensemble # ансамбли
from sklearn import metrics # метрики
from sklearn.model_selection import train_test_split # сплитование выборки
from sklearn.model_selection import cross_val_score, StratifiedKFold # кросс-валидация

from sklearn.model_selection import GridSearchCV # подбор гиперпараметров с помощью GridSearchCV
from sklearn.model_selection import RandomizedSearchCV # подбор гиперпараметров с помощью RandomizedSearchCV
from hyperopt import hp, fmin, tpe, Trials # подбор гиперпараметров с помощью Hyperopt
import optuna # подбор гиперпараметров с помощью Optuna

In [2]:
# загружаем данные
data = pd.read_csv('data/train_sem.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# создаем матрицу наблюдений X и вектор ответов y
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [4]:
# разделяем выборку на тренировочную и тестовую в соотношении 80/20 (4 признака /1 признак )
# для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение). 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

### <center> **Модель логистическая регрессия** 

In [5]:
# создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(
    random_state=42,
    max_iter = 1000
    )

# обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

# выведем значение метрики
print("accuracy на тестовом наборе: {:.2f}".format(log_reg.score(X_test, y_test)))
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78


### <center> **Подбор гиперпараметров с помощью GridSearchCV**

In [6]:
# зададим словарь значений гиперпараметров (сетку)
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
               'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.1, 0.2, 0.3, 0.5, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
               'solver': ['liblinear', 'saga'],
               'C': [0.1, 0.2, 0.3, 0.5, 1]}
]

# создадим объект GridSearchCV, передав в него модель -
# логистическую регрессию, список словарей значений гиперпараметров,
# задав 5 фолдов в кросс-валидации и использование всех ядер процессора
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state = 42, # генератор случайных чисел
        max_iter = 1000 # количество итераций на сходимость
    ), 
    param_grid = param_grid, 
    cv = 5, 
    n_jobs = -1
)  

# оптимизируем гиперпараметры при заданных условиях
%time grid_search.fit(X_train, y_train) 

# выведем результаты оптимизации
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test, y_test)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 40.3 s
Wall time: 6min 28s
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'C': 0.2, 'penalty': 'l1', 'solver': 'saga'}


### <center> **Подбор гиперпараметров с помощью RandomizedSearchCV**

In [7]:
# зададим словарь значений гиперпараметров (множество вариантов выбора)
param_distributions = [
                       {'penalty': ['l2', 'none'] , # тип регуляризации
                        'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
                        'C': [0.1, 0.2, 0.3, 0.5, 1]}, # уровень силы регурялизации
              
                       {'penalty': ['l1', 'l2'] ,
                        'solver': ['liblinear', 'saga'],
                        'C': [0.1, 0.2, 0.3, 0.5, 1]}
]

# создадим объект RandomizedSearchCV, передав в него модель -
# логистическую регрессию, список словарей значений гиперпараметров,
# задав 5 фолдов в кросс-валидации, количество итераций 10 и использование всех ядер процессора            
random_search = RandomizedSearchCV(
    estimator = linear_model.LogisticRegression(
        random_state = 42, # генератор случайных чисел
        max_iter = 1000 # количество итераций на сходимость    
    ),  
    param_distributions = param_distributions, 
    cv = 5, 
    n_iter = 10, 
    n_jobs = -1
)  

# оптимизируем гиперпараметры при заданных условиях
%time random_search.fit(X_train, y_train) 

# выведем результаты оптимизации
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 44.3 s
Wall time: 2min 52s
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.5}


C:\Users\artem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### <center> **Модель случайный лес** 

In [8]:
# создаем объект класса дерево решений
rf = ensemble.RandomForestClassifier(random_state=42)

# обучаем дерево по алгоритму CARTrint
rf.fit(X_train, y_train)

# выводим значение метрики 
print("accuracy на тестовом наборе: {:.2f}".format(rf.score(X_test, y_test)))
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.80


### <center> **Подбор гиперпараметров с помощью Hyperopt**

In [18]:
# зададим пространство поиска гиперпараметров
space = {'n_estimators': hp.quniform('n_estimators', 50, 200, 1), # количество деревьев
         'max_depth' : hp.quniform('max_depth', 10, 20, 1), # максимальная глубина дерева
         'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 15, 1) # минимальное число объектов в листе
        }

In [19]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    
    # применим кросс-валидацию с переданным в функцию 
    # количеством фолдов и возьмем среднее значение результатов
    # вычисления метрики f1
    
    # задаём параметры кросс-валидации (стратифицированная с перемешиванием)
    skf = StratifiedKFold(n_splits=cv, 
                          shuffle=True, 
                          random_state=1
                         )
    # проводим кросс-валидацию 
    score = cross_val_score(model, X, y, cv=skf, scoring="f1", n_jobs=-1).mean()

    # ставим знак минус т.к. hyperopt минимизирует,
    # а нам нужен максимум значения метрики
    return -score

In [20]:
%%time
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best = fmin(hyperopt_rf, # наша функция 
            space = space, # пространство гиперпараметров
            algo = tpe.suggest, # алгоритм оптимизации, по умолчанию
            max_evals = 20, # максимальное количество итераций
            trials = trials, # логирование результатов
            rstate=np.random.default_rng(42) # фиксируем для повторяемости результата
           )

print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 20/20 [00:53<00:00,  2.66s/trial, best loss: -0.8144022224940158]
Наилучшие значения гиперпараметров {'max_depth': 13.0, 'min_samples_leaf': 3.0, 'n_estimators': 125.0}
CPU times: total: 23.1 s
Wall time: 53.2 s


In [21]:
# рассчитаем метрику для тестовой выборки построив модель
# c наилучшими значниями гиперпараметров
model = ensemble.RandomForestClassifier(
    random_state = 42, # фиксируем для повторяемости результата
    n_estimators = int(best['n_estimators']),
    max_depth = int(best['max_depth']),
    min_samples_leaf = int(best['min_samples_leaf'])
)

# обучаем модель 
model.fit(X_train, y_train)

# выводим значение метрики 
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.80


### <center> **Подбор гиперпараметров с помощью Optuna**

In [16]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 50, 200, 1) # количество деревьев
  max_depth = trial.suggest_int('max_depth', 10, 20, 1) # максимальная глубина дерева
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1) # минимальное число объектов в листе

  # создаем модель
  model = ensemble.RandomForestClassifier(
      n_estimators = n_estimators,
      max_depth = max_depth,
      min_samples_leaf = min_samples_leaf,
      random_state = 42)
    
  # обучаем модель
  model.fit(X_train, y_train)
    
  # применим кросс-валидацию с количеством фолдов 5 
  # и возьмем среднее значение результатов вычисления метрики f1
    
  # задаём параметры кросс-валидации (стратифицированная с перемешиванием)
  skf = StratifiedKFold(n_splits=5, 
                        shuffle=True, 
                        random_state=1
                       )
  # проводим кросс-валидацию 
  score = cross_val_score(model, X_train, y_train, cv=skf, scoring="f1", n_jobs=-1).mean()  

  return score

In [17]:
%%time
# cоздаем объект исследования,указав максимизацию метрики
study = optuna.create_study(study_name = "RandomForestClassifier", 
                            direction = "maximize"
                            )

# ищем лучшую комбинацию гиперпараметров 20 раз
study.optimize(optuna_rf, n_trials=20)

print("Наилучшие значения гиперпараметров {}".format(study.best_params))

[I 2023-06-23 04:09:02,014] A new study created in memory with name: RandomForestClassifier
[I 2023-06-23 04:09:05,599] Trial 0 finished with value: 0.8135672740728541 and parameters: {'n_estimators': 145, 'max_depth': 12, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8135672740728541.
[I 2023-06-23 04:09:08,237] Trial 1 finished with value: 0.8128410790781586 and parameters: {'n_estimators': 93, 'max_depth': 18, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8135672740728541.
[I 2023-06-23 04:09:09,802] Trial 2 finished with value: 0.7989231567628704 and parameters: {'n_estimators': 52, 'max_depth': 18, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.8135672740728541.
[I 2023-06-23 04:09:13,978] Trial 3 finished with value: 0.8076045292328127 and parameters: {'n_estimators': 193, 'max_depth': 12, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.8135672740728541.
[I 2023-06-23 04:09:16,806] Trial 4 finished with value: 0.8073651219125347 and parameters: {'n_es

Наилучшие значения гиперпараметров {'n_estimators': 84, 'max_depth': 16, 'min_samples_leaf': 2}
CPU times: total: 25.7 s
Wall time: 58.4 s


In [15]:
# рассчитаем метрику для тестовой выборки построив модель
# c наилучшими значниями гиперпараметров
model = ensemble.RandomForestClassifier(**study.best_params,random_state=42)

# обучаем модель 
model.fit(X_train, y_train)

# выводим значение метрики 
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
